# NLP

> author분류
>
> 

사전 학습 모델의 사용은 금지이며
가중치가 모두 초기화 된 모델의 '구조'만을 사용하여 제공된 데이터를 이용한 학습은 가능합니다.

사전학습 된 BERT를 대회 데이터로 추가 학습 (fine-tuning)하지 않고 임베딩 레이어와 같이 사용

bert가 안된다고 하신적이 없는 것같은데... fasttext나 glove에서 사전학습 임베딩 데이이터를 사용하는 건 되고,  bert를 사용하는 건 안된다고 명시된 것도 아닌데 그냥 임베딩 단계까지 알아서 사용하면 되는 것 아닐까요?


!

In [1]:
!pip install pyLDAvis

  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\User\\miniconda3\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [1]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [62]:
#파일 불러오기
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test_x.csv', encoding = 'utf-8')
#sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [63]:
train.head()
print(train.shape)
print(test.shape)

(54879, 3)
(19617, 2)


In [64]:
train=train.drop('index',axis=1)
test=test.drop('index',axis=1)

In [65]:
train.head(5)

,text,author
0,"He was almost choking. There was so much, so m...",3
1,"“Your sister asked for it, I suppose?”",2
2,"She was engaged one day as she walked, in per...",1
3,"The captain was in the porch, keeping himself ...",4
4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [66]:
test.head(5)

,text
0,“Not at all. I think she is one of the most ch...
1,"""No,"" replied he, with sudden consciousness, ""..."
2,As the lady had stated her intention of scream...
3,“And then suddenly in the silence I heard a so...
4,His conviction remained unchanged. So far as I...


# 자연어 전처리

- preprocessing
    1) 부호 제거 
    2) 불용어 제거> stopwords removing
    3) 형태소 분석> stemming ** 중요>> 토큰화
    4) 표제어 추출> lemmatization
- vectorization ** 정수 인코딩으로 변환
    1) one-hot encoding
    2) count vectorization
    3) tf-idf
    4) padding * 길이 조정
   
- embedding *벡터들을 특정 차원으로 임베딩 
    1) word2vec
    2) doc2vec
   
- modeling
    1) GRU
    2) LSTM
   

# 1) preprocessing

## 1-1) 부호제거


In [67]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [68]:
#부호가 사라진 것을 확인할 수 있다.
train['text']

0        He was almost choking There was so much so muc...
1                       Your sister asked for it I suppose
2         She was engaged one day as she walked in peru...
3        The captain was in the porch keeping himself c...
4        Have mercy gentlemen odin flung up his hands D...
                               ...                        
54874    Is that you Mr Smith odin whispered I hardly d...
54875    I told my plan to the captain and between us w...
54876     Your sincere wellwisher friend and sister LUC...
54877                 Then you wanted me to lend you money
54878    It certainly had not occurred to me before but...
Name: text, Length: 54879, dtype: object

In [70]:
pip install Okt

ERROR: Could not find a version that satisfies the requirement oktapysdk (from okt) (from versions: none)
ERROR: No matching distribution found for oktapysdk



  Using cached okt-0.0.3-py3-none-any.whl (4.5 kB)


In [71]:
train.info()
## 타입을 str으로 바꾸기

train['text']=train['text'].astype(str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54879 entries, 0 to 54878
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    54879 non-null  object
 1   author  54879 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 857.6+ KB


In [72]:
train['text'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 54879 entries, 0 to 54878
Series name: text
Non-Null Count  Dtype 
--------------  ----- 
54879 non-null  object
dtypes: object(1)
memory usage: 428.9+ KB


In [73]:
# from konlpy.tag import Okt
# Okt = Okt()

# sentence = '데이콘에서 다양한 컴피티션을 즐기면서 실력있는 데이터 분석가로 성장하세요!!.'

# print("형태소 단위로 문장 분리")
# print("----------------------")
# print(Okt.morphs(sentence))
# print(" ")
# print("문장에서 명사 추출")
# print("----------------------")
# print(Okt.nouns(sentence))
# print(" ")
# print("품사 태킹(PoS)")
# print("----------------------")
# print(Okt.pos(sentence))

형태소 단위로 문장 분리
----------------------
['데', '이콘', '에서', '다양한', '컴피티션', '을', '즐기면서', '실력', '있는', '데이터', '분석', '가로', '성장하세요', '!!.']
 
문장에서 명사 추출
----------------------
['데', '이콘', '컴피티션', '실력', '데이터', '분석', '가로']
 
품사 태킹(PoS)
----------------------
[('데', 'Noun'), ('이콘', 'Noun'), ('에서', 'Josa'), ('다양한', 'Adjective'), ('컴피티션', 'Noun'), ('을', 'Josa'), ('즐기면서', 'Verb'), ('실력', 'Noun'), ('있는', 'Adjective'), ('데이터', 'Noun'), ('분석', 'Noun'), ('가로', 'Noun'), ('성장하세요', 'Adjective'), ('!!.', 'Punctuation')]



## 1-2) 불용어 제거


In [74]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "no","not","odin","said","will","now","about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [75]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [76]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])


In [77]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly not occurred said yes like'],
      dtype='<U1433')


## 1-3) 형태소 분석
- stemming ** 중요>> 토큰화

In [78]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

author_list={}

for idx,row_data in train.iterrows():
    author=row_data['author']# 작가 번호 
    setence=row_data['text']# text
    
    if author not in author_list:
        author_list[author]=[]
    author_list[author].append(setence)
        

author_list
#tokenize해서 빈도를 시각화하기    
from nltk.probability import FreqDist

frequencies = {}

for author,sentences in author_list.items():
    all_words=[]
    for sentence in sentences:
        words = word_tokenize(sentence)  # 문장을 단어로 토큰화
        all_words+=words
        
    fdist=FreqDist(all_words)
    frequencies[author] = fdist



In [79]:
# 작가별로 가장 많이 사용된 단어 출력

for author, fdist in frequencies.items():
    print(f"Author: {author}")
    print(f"Most common words: {fdist.most_common(10)}")  # 상위 10개 단어 출력
    print("\n")
    

Author: 3
Most common words: [('odin', 11542), ('not', 5533), ('one', 2401), ('said', 2336), ('no', 2220), ('will', 1756), ('know', 1617), ('now', 1587), ('dont', 1506), ('come', 1366)]


Author: 2
Most common words: [('odin', 4625), ('not', 2651), ('said', 2473), ('upon', 2101), ('one', 1765), ('no', 1713), ('man', 1362), ('will', 1239), ('can', 1089), ('us', 1057)]


Author: 1
Most common words: [('odin', 8479), ('not', 5239), ('no', 1845), ('mr', 1742), ('said', 1491), ('will', 1356), ('mrs', 1330), ('odins', 1219), ('must', 1133), ('miss', 1082)]


Author: 4
Most common words: [('odin', 3981), ('said', 2388), ('not', 2124), ('upon', 1212), ('no', 1065), ('man', 1056), ('one', 1014), ('will', 893), ('now', 888), ('like', 775)]


Author: 0
Most common words: [('odin', 7509), ('said', 4755), ('mr', 3173), ('not', 2985), ('no', 1804), ('one', 1361), ('little', 1183), ('upon', 1137), ('know', 1096), ('now', 1041)]




In [80]:
#빈도 객체를 데이터 프레임으로 변경
import matplotlib.pyplot as plt
import seaborn as sns

data = []
for author, fdist in frequencies.items():
    for word, freq in fdist.items():
        data.append({'Author': author, 'Word': word, 'Frequency': freq})
df = pd.DataFrame(data)



In [85]:
sorted_df=df.sort_values(by='Frequency',ascending=False)
sorted_df.head(20)


,Author,Word,Frequency
16,3,odin,11542
40006,1,odin,8479
73014,0,odin,7509
38,3,not,5533
40000,1,not,5239
72989,0,said,4755
19295,2,odin,4625
55307,4,odin,3981
73015,0,mr,3173
72985,0,not,2985


In [86]:
plt.figure(figsize=(16,10))
sns.barplot(x='Author', y='Frequency', hue='Word', data=df)
# plt.legend(title='Word')
# plt.title('Word Frequencies by Author')
# plt.xlabel('Author')
# plt.ylabel('Frequency')
#plt.show()

## 1-4) n-gram (2, trie)

In [1]:
def generate_ngrams(text, n):
    words = text.split()  # 입력 텍스트를 단어로 분리
    ngrams = []
    
    for i in range(len(words) - n + 1):
        ngram = ' '.join(words[i:i+n])  # n개의 단어를 연결하여 n-gram 생성
        ngrams.append(ngram)
    
    return ngrams

input_text = "I love natural language processing"
bigrams = generate_ngrams(input_text, 2)  # 바이그램 생성
trigrams = generate_ngrams(input_text, 3)  # 트라이그램 생성

print("Bigrams:", bigrams)
print("Trigrams:", trigrams)


Bigrams: ['I love', 'love natural', 'natural language', 'language processing']
Trigrams: ['I love natural', 'love natural language', 'natural language processing']


# 2) vectorization

#  모델링

In [18]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [19]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index


In [20]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [21]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [22]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 16)           320000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 5)                 125       
                                                                 
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 16s - loss: 1.5662 - accuracy: 0.2742 - val_loss: 1.5505 - val_accuracy: 0.2684 - 16s/epoch - 12ms/step
Epoch 2/20
1372/1372 - 15s - loss: 1.4357 - accuracy: 0.3854 - val_loss: 1.2980 - val_accuracy: 0.4463 - 15s/epoch - 11ms/step
Epoch 3/20
1372/1372 - 15s - loss: 1.1942 - accuracy: 0.4868 - val_loss: 1.1694 - val_accuracy: 0.4856 - 15s/epoch - 11ms/step
Epoch 4/20


ResourceExhaustedError: Graph execution error:

Detected at node 'AssignAddVariableOp_3' defined at (most recent call last):
    File "c:\Users\User\miniconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\User\miniconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\User\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\User\miniconda3\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\User\miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\User\miniconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\User\miniconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()


In [ ]:
# predict values
pred = model.predict_proba(test_padded)

In [ ]:
pred 

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

In [ ]:
#sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')

#0.58012

pip install torchtext

In [ ]:
import re
from copy import deepcopy



import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data





batch_size = 32
embedded_size = 128# ?
hidden_size = 150
n_classes = 5
EPOCHS = 7


TEXT = data.Field(sequential=True,

         use_vocab=True,

         tokenize=str.split, 

         lower=True,

         batch_first=True,

         fix_length=None)



LABEL = data.Field(sequential=False,

          use_vocab=False,

          is_target=True)



from torchtext.data import TabularDataset



train_data = TabularDataset(

  path = './data/train.nltk.csv',

  format = 'csv',

  fields = [('text', TEXT), ('label', LABEL)],

  skip_header = True)





test_data = TabularDataset(

  path = './data/test_x.nltk.1.csv',

  format = 'csv',

  fields = [('text', TEXT)],

  skip_header = True)



print(vars(train_data[0]))

TEXT.build_vocab(train_data, min_freq=7, max_size=20000, vectors = "fasttext.en.300d")

train_data, valid_data = train_data.split(split_ratio=0.8, stratified=True)



from torchtext.data import BucketIterator



train_loader, valid_loader = BucketIterator.splits(

  (train_data, valid_data),

  batch_size = batch_size,

  device='cuda:0',

  shuffle = True,

  sort_key=lambda x:len(x.text),

  sort_within_batch=True

)



test_loader = BucketIterator(

  test_data,

  batch_size = batch_size,

  device='cuda:0',

  shuffle = False,

  sort_key=lambda x:len(x.text),

  sort_within_batch=True

)





from tqdm import tqdm

model = LSTM(len(TEXT.vocab), embedded_size, hidden_size, n_classes).to('cuda:0')



crit = nn.NLLLoss().to('cuda:0')

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_loss = 999999

best_model = None



for i in range(EPOCHS):

  train_loss = 0

  train_accuracy = 0

  model.train()

  for x, y in tqdm(train_loader):

    optimizer.zero_grad()

    x, y = x.to('cuda:0'), y.to('cuda:0')



    y_hat = model(x)

    loss = crit(y_hat, y)

    loss.backward()

    optimizer.step()

    if isinstance(y, torch.LongTensor) or isinstance(y, torch.cuda.LongTensor):

      accuracy = (torch.argmax(y_hat, dim=-1) == y).sum() / float(y.size(0))

    else:

      accuracy = 0

    train_loss += float(loss) / len(train_loader)

    train_accuracy += accuracy / len(train_loader)



  valid_loss = 0

  valid_accuracy = 0

  with torch.no_grad():

    model.eval()

    for x, y in valid_loader:

      x, y = x.to('cuda:0'), y.to('cuda:0')

      y_hat = model(x)

      loss = crit(y_hat, y)

       

      if isinstance(y, torch.LongTensor) or isinstance(y, torch.cuda.LongTensor):

        accuracy = (torch.argmax(y_hat, dim=-1) == y).sum() / float(y.size(0))

      else:

        accuracy = 0

      valid_loss += float(loss) / len(valid_loader)

      valid_accuracy += accuracy / len(valid_loader)

       

    if valid_loss < best_loss:

      best_loss = valid_loss

      best_model = deepcopy(model.state_dict())

       

       

  print("EPOCHS: {:2d} | train_accuracy: {:.4f} / train_loss: {:.4f} / valid_accuracy: {:.4f} / valid_loss: {:.4f} / best_loss: {:.4f}".format

       (i+1, train_accuracy, train_loss, valid_accuracy, valid_loss, best_loss))



test_model = LSTM(len(TEXT.vocab), embedded_size, hidden_size, n_classes).to('cuda:0')

test_model.load_state_dict(best_model)



test_model.eval()

y_hats = []

with torch.no_grad():

  for x_batch in test_loader:

    x = x_batch.text.to('cuda:0')

    y_hat = test_model(x).cpu()

    y_hats += y_hat

  y_hats = torch.stack(y_hats).exp()

test_pred = y_hats.numpy()



import pandas as pd

sub = pd.read_csv('./data/sample_submission.csv', index_col=0)



sub[sub.columns] = test_pred

sub.head()

sub.to_csv('./data/submission2.csv')